<a href="https://colab.research.google.com/github/SELOMELO280104/Roboflow_test/blob/main/polygon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ayakkabı Segmentasyon Projesi (YOLOv8)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SELOMELO280104/Roboflow_test/blob/main/polygon.ipynb)

Polygon Test

In [ ]:
# Gerekli kütüphaneleri kur
!pip install ultralytics -q
import os
import zipfile
from google.colab import files
from ultralytics import YOLO

# 1. Zip dosyasını yükleme ekranı
print("👇 Lütfen Roboflow'dan indirdiğiniz '.zip' uzantılı veri setini seçin:")
uploaded = files.upload()

if uploaded:
    zip_filename = list(uploaded.keys())[0]
    extract_dir = '/content/dataset_bbox'

    # 2. Zip dosyasını çıkartma
    print(f"\n📦 '{zip_filename}' çıkartılıyor...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # 3. data.yaml dosyasını otomatik bulma
    yaml_path = None
    for root, dirs, dosyalar in os.walk(extract_dir):
        if 'data.yaml' in dosyalar:
            yaml_path = os.path.join(root, 'data.yaml')
            break

    if yaml_path:
        print(f"✅ Veri seti hazır! Dosya yolu: {yaml_path}")

        # 4. Eğitimi Başlatma (Bounding Box için yolov8n.pt)
        print("\n🚀 Bounding Box (Kutu) eğitimi T4 GPU üzerinde başlıyor...")
        model = YOLO('yolov8n.pt')

        results = model.train(
            data=yaml_path,
            epochs=30,
            imgsz=640,
            batch=16,
            project='/content/Trained_model',
            name='ayakkabi_bbox_sonuc'
        )
        print("\n🎉 Eğitim başarıyla tamamlandı!")
    else:
        print("❌ HATA: Zip dosyasının içinde 'data.yaml' bulunamadı!")
else:
    print("❌ Dosya yüklemesi iptal edildi.")

In [ ]:
from IPython.display import Image, display
import glob
from ultralytics import YOLO

print("🔍 Test verileriyle performans ölçülüyor...")

# Eğitilmiş en iyi modeli yükle
best_model_path = '/content/Trained_model/ayakkabi_bbox_sonuc/weights/best.pt'
model = YOLO(best_model_path)

# Sadece 'test' klasöründeki resimleri kullanarak doğrulama yap
metrics = model.val(split='test', project='/content/Trained_model', name='test_sonuclari')

# Test sonuçlarından örnek bir görseli ekrana bas
print("\n📷 Test Klasöründen Örnek Tespit:")
resimler = glob.glob('/content/Trained_model/test_sonuclari*/val_batch0_pred.jpg')

if resimler:
    display(Image(filename=resimler[0]))
else:
    print("Örnek görsel bulunamadı.")

In [ ]:
from google.colab import files

best_model_path = '/content/Trained_model/ayakkabi_bbox_sonuc/weights/best.pt'
print("⬇️ Eğitilmiş model (best.pt) bilgisayarınıza indiriliyor...")

try:
    files.download(best_model_path)
    print("✅ İndirme işlemi başlatıldı!")
except Exception as e:
    print(f"❌ İndirme sırasında bir hata oluştu: {e}")